<a href="https://colab.research.google.com/github/nunocesarsa/CML_Short_tutorial/blob/master/AutoML/AutoSklearn/Tutorial_AutoSklearn_RemoteSensing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing AutoSklearn

- has a very hard installation... bear with me
- distributed has to be removed and reinstalled
- And that implies restarting the runtime 

https://automl.github.io/auto-sklearn/master/

## Notice:
 you will have to restart the runtime (button appears in the cell) during the next cell

In [ ]:
#it is a hard install on jupyter notebook
!pip install PipelineProfiler

!apt-get install swig -y
!pip install Cython numpy
!sudo apt-get install build-essential swig

import distributed
print(distributed.__version__)

!pip uninstall distributed
!pip install distributed

import distributed
print(distributed.__version__)

In [ ]:
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install 

In [ ]:
!pip install auto-sklearn==0.11.0 #weirdly this might require multiple attempts, something crashes in pyrfr or elsewhere. ALSO installing an earlier version because latest update crashes

In [ ]:
#to investigate outputs of autosklearn
import sklearn

import autosklearn

##Example 

In [ ]:
#Example

import autosklearn.classification
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics


if __name__ == "__main__":
  X, y = sklearn.datasets.load_digits(return_X_y=True)
  X_train, X_test, y_train, y_test = \
          sklearn.model_selection.train_test_split(X, y, random_state=1)
  automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=60)
  automl.fit(X_train, y_train)
  y_hat = automl.predict(X_test)
  print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_hat))

### Exploring the pipelines

In [ ]:
import PipelineProfiler
prof_data = PipelineProfiler.import_autosklearn(automl)
PipelineProfiler.plot_pipeline_matrix(prof_data)

# Setting up data 

## Downloading

The data is "stolen" from this excellent tutorial on how to do a classification with R:

*   https://urbanspatial.github.io/classifying_satellite_imagery_in_R/
*   https://github.com/urbanSpatial/classifying_satellite_imagery_in_R




In [ ]:
! git clone https://github.com/urbanSpatial/classifying_satellite_imagery_in_R

## Exploring

In [ ]:
!pip install rasterio
!pip install geopandas
!pip install earthpy
!pip install rasterstats

In [9]:
import os
from glob import glob
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import plotting_extent
import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

import rasterstats


## Checking out the data

In [ ]:
#From: https://earthpy.readthedocs.io/en/latest/gallery_vignettes/plot_rgb.html

#fetches the iamges at 30m excluding the panchromatic and cirrus 
landsat_bands_data_path = "/content/classifying_satellite_imagery_in_R/data/band*[1-7]*.tif"

stack_band_paths = glob(landsat_bands_data_path)
stack_band_paths.sort()

#resorting because of the band names 
stack_band_paths_sorted = [stack_band_paths[i] for i in [0,3,4,5,6,7,8]]

#print(stack_band_paths)
#print(stack_band_paths_sorted)

# Create image stack and apply nodata value for Landsat
arr_st, meta = es.stack(stack_band_paths_sorted, nodata=-9999)

#From: https://earthpy.readthedocs.io/en/latest/gallery_vignettes/plot_rgb.html

# Create figure with one plot
fig, ax = plt.subplots(figsize=(12, 12))

ep.plot_rgb(arr_st, rgb=(3, 2, 1), ax=ax, title="Landsat 8 RGB Image")
plt.show()

##Loading and ploting the points

In [ ]:
#Loading and reprojecting
sample_shp = gpd.read_file('/content/classifying_satellite_imagery_in_R/data/calgary_trainingPoints.shp')
shp_prj = sample_shp.to_crs(epsg=32612)
#shp_prj.crs
shp_prj

In [ ]:
#we have to fetch the extent using the rasterio package
with rio.open('/content/classifying_satellite_imagery_in_R/data/band1.tif') as image_src:
  img_data = image_src.read()

  img_extent = plotting_extent(image_src)
  



fig, ax = plt.subplots(figsize=(12, 12))

ep.plot_rgb(arr_st, rgb=(3, 2, 1), ax=ax, title="Landsat 8 RGB Image",extent=(img_extent))
shp_prj.plot(ax=ax)
plt.show()

#Extracting values

In [ ]:
#first we save everything (the stacked raster and the projected shapefile to a directory)
!mkdir outputs

#saving
shp_prj.to_file('/content/outputs/sample_loc.shp')
es.stack(stack_band_paths_sorted, out_path='/content/outputs/Landsat.tif') #fails because of uint16 if we add the thermals

#other alternatives here: https://gis.stackexchange.com/questions/223910/using-rasterio-or-gdal-to-stack-multiple-bands-without-using-subprocess-commands

### Actual loop for extracting values... grrr

In [ ]:
from rasterstats import point_query

for i in range(1,8):
  #pt_query = point_query('/content/outputs/sample_loc.shp',i)
  #print(pt_query[0])

  #querying the data
  pt_query = point_query('/content/outputs/sample_loc.shp','/content/outputs/Landsat.tif',band=i)
  #print(i)
  #print(pt_query[0])

  band_nr = "B"+str(i)
  print('Processing:',band_nr)

  #adding to the pandas
  shp_prj[band_nr]=pt_query

# Setting up for Autosklearn




### Partitioning the data

In [ ]:
shp_prj
#notice the units of the bands should be converted to reflectance but it doesnt really matter for machine learning

In [20]:
#removing NA from the datasets
test_data = shp_prj
test_data = test_data.dropna(0) #there is one row somewhere with an NA that causes errors, se we just remove it

#selecting data and conerting to category
df_x = test_data[["B1","B2","B3","B4","B5","B6","B7"]]
df_y = test_data[['class']].astype('category')

#using Sklearn data splitter
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(df_x, df_y, random_state=42,test_size=0.3) #.7/.3 split

In [ ]:
#feel free to explore the datasets here
y_train

## Applying AutoSklearn to the RS data

Find more options for Autosklearn @ the API page: https://automl.github.io/auto-sklearn/master/api.html


- Autosklearn can be run using a "meta-learning" approach: meaning, the first N models tested are a set of pre-trained pipelines tested on other data which are "similar" to the current dataset

- "Simulates" the expertise of a user who would have "seen data that looks like this before".

In [ ]:
time_spent = 60 #total time spent trying different models, the more time, the more models are tested

#this command constructs the object auto_mdl, an autosklearn insteance
auto_mdl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=time_spent,
                                                       resampling_strategy='cv',
                                                       resampling_strategy_arguments= {'folds': 5},
                                                       seed=42,
                                                       n_jobs=-1)

#this command actually performs the fit
auto_mdl.fit(X_train,y_train)

In [ ]:
y_hat = auto_mdl.predict(X_train)
print("Training accuracy score", sklearn.metrics.accuracy_score(y_train, y_hat))

y_hat_vl = auto_mdl.predict(X_test)
print("Validation accuracy score", sklearn.metrics.accuracy_score(y_test, y_hat_vl))


## Investigating the pipeline:

- this can be used to identify "common" pre-processing steps that improve the accuracy
- Can also be used to identify specific algorithms that work better 
- And naturally, all this knowledge can be encoded to a custom autosklearn instance

In [ ]:
prof_data = PipelineProfiler.import_autosklearn(auto_mdl)
PipelineProfiler.plot_pipeline_matrix(prof_data)

#Predicting on a raster

- I reckon this can be simplified but i adapted from a crappy code i used for testing in another case

1.   List item
2.   List item



## Preparing the data

- In general: Convert the multy-layer raster object to a "table format", apply the model and reconstruct the raster. 

- (Optional) This procedure above allows me to process in data chunks and to avoid going out of memory on the google colab

In [41]:
!pip install pyrsgis
from pyrsgis.convert import changeDimension
from pyrsgis import raster
import rasterio
import pyproj
import numpy as np
import pandas as pd


#lading the landsat data 
ds1, bands = raster.read('/content/outputs/Landsat.tif')

print(ds1)
print(bands.shape) 

(7, 1413, 1121)


In [ ]:
#creates a np with 7 columns anx NxM rows (and also its tranposed)

bandByPixel = changeDimension(bands) #we have to devide all values by 10k - its a conversion from bits to reflectances
bandByPixel_t = np.transpose(bandByPixel)

print(bandByPixel.shape)
print(bandByPixel_t.shape)

pd_bandByPixel = pd.DataFrame(data=bandByPixel,columns=["B1","B2","B3","B4","B5","B6","B7"])

In [ ]:
pd_bandByPixel #notice that many of the lines are 0, because, the original raster is padded by 0'

## Prediction time

- be ready to wait a bit... 

In [ ]:
y_pred = auto_mdl.predict(pd_bandByPixel)
y_pred

## Rebuilding the raster

## Converting "strings" to numbers

In [ ]:
#checking the clases
shp_prj['class'].unique()

In [60]:
#lazy but i don't care:
y_pred_num = np.where(y_pred=='water', 0, y_pred) 
y_pred_num = np.where(y_pred_num=='undeveloped', 1, y_pred_num) 
y_pred_num = np.where(y_pred_num=='developed', 2, y_pred_num) 
y_pred_num = np.where(y_pred_num=='clouds', 3, y_pred_num) 

In [ ]:
#rebuilds the raster from the table we used before
y_pred_np = np.reshape(y_pred_num,(ds1.RasterYSize,ds1.RasterXSize))
y_pred_np.shape



##Saving the raster

In [62]:
#exports the data to our work space
raster.export(y_pred_np, ds1, '/content/outputs/Landsat_Class.tif', dtype='int') 